# Projeto Final - T319 (2S2025)

### Instruções

1. Quando você terminar os exercícios do projeto, vá até o menu do Colab ou Jupyter e selecione a opção para fazer download do notebook.
    * Os notebooks tem extensão .ipynb.
    * Este deve ser o arquivo que você irá entregar.
    * No Colab vá até a opção **File** -> **Download .ipynb**.
    * No Jupyter vá até a opção **File** -> **Download as** -> **Notebook (.ipynb)**.
2. Após o download do notebook, vá até a aba de tarefas do MS Teams, localize a tarefa referente a este projeto e faça o upload do seu notebook. Veja que há uma opção para anexar arquivos à tarefa.
3. Atente-se ao prazo de entrega definido na tarefa do MS Teams. Entregas fora do prazo não serão consideradas.
4. **O projeto pode ser resolvido em grupos de no MÁXIMO 3 alunos**.
5. Todas as questões têm o mesmo peso.
6. Questões copiadas de outros grupos serão anuladas em todos os grupos com a mesma resposta.
7. Não se esqueça de colocar seu(s) nome(s) e número(s) de matrícula no campo abaixo. Coloque os nomes dos integrantes do grupo no campo de texto abaixo.
8. Você pode consultar todo o material de aula e laboratórios.
9. A interpretação faz parte do projeto. Leia o enunciado de cada questão atentamente!
10. Boa sorte!

**Nomes e matrículas**:

1. Nome do primeiro aluno - Matrícula do primeiro aluno
2. Nome do segundo aluno - Matrícula do segundo aluno
3. Nome do terceiro aluno - Matrícula do terceiro aluno

## Exercícios

### 1) Exercício sobre a escolha do passo de aprendizagem

1. Execute a célula de código abaixo para importar as bibliotecas e definir algumas funções necessárias para o treinamento de um modelo de regressão linear.

**DICAS**

+ A função `gradientDescent` implementa a versão **estocástica** do gradiente descendente.
+ Note que a função `gradientDescent` utiliza **decaimento temporal** do passo de aprendizagem para tornar o aprendizado do algoritmo mais comportado.

In [ ]:
# Import all necessary libraries.
import random
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# Reseta os gerados de sequências pseudo-aleatórias.
seed = 42
np.random.seed(seed)
random.seed(seed)

def calculateErrorSurface(x, y):
    """Generate data points for plotting the error surface."""

    # Retrieve number of examples.
    N = len(y)

    # Generate values for parameter space.
    M = 200
    a1 = np.linspace(2.5, 2.59, M)
    a2 = np.linspace(-1.020, -0.995, M)

    # Generate matrices with combinations between a0 and a1 values.
    A1, A2 = np.meshgrid(a1, a2)

    # Generate points for plotting the cost-function surface.
    J = np.zeros((M,M))
    for iter1 in range(0, M):
        for iter2 in range(0, M):
            # Hypothesis function (a second degree function).
            yhat = A1[iter1, iter2]*x + A2[iter1, iter2]*np.floor(2.5*x + 0.5)
            # Calculate the mean squared error (MSE) for each pair of values.
            J[iter1, iter2] = (1.0/N)*np.sum(np.square(y - yhat))

    return J, A1, A2

def timeBasedDecay(alpha_init, k, t):
    '''Decaimento temporal.'''
    return alpha_init / (1.0 + k*t)

def gradientDescent(X_train, y_train, X_test, y_test, n_epochs, alpha_init, k):
    '''
    Função que implementa a versão estocástica do gradiente descendente.
    Os parâmetros de entrada da função são:
    * X_train    - Matriz de atributos de treinamento
    * y_train    - vetor de rótulos de treinamento
    * X_test     - Matriz de atributos de teste
    * y_test     - vetor de rótulos de teste
    * n_epochs   - número máximo de épocas de treinamento
    * alpha_init - valor inicial do passo de aprendizagem
    * k          - taxa de decaimento da redução temporal do passo de aprendizagem

    Os valores retornados são:
    * a               : vetor de pesos correspondente à última iteração de atualização
    * a_min           : vetor de pesos correspondente ao menor erro de teste
    * Jgd             : vetor com os valores do erro de treinamento ao longo do treinamento
    * Jgd_test        : vetor com os valores do erro de teste ao longo do treinamento
    * a_hist          : matriz com o histórico dos valore do vetor de pesos
    * alpha_hist      : vetor com histórico dos valores do passo de aprendizagem
    * update_hist     : matriz com o histórico de vetores de atualização dos pesos
    * gradient_hist   : matriz com o histórico de vetores gradiente
    * iteration       : valor da última iteração
    '''

    # Reseta os geradores de sequências pseudo-aleatórias.
    np.random.seed(seed)
    random.seed(seed)

    # Number of training examples.
    N_train = len(y_train)

    # Number of test examples.
    N_test = len(y_test)

    # Reshape y to be a column vector.
    y_train = y_train.reshape(N_train,1)

    # Inicialização do vetor de pesos.
    a = np.array([-5.0, -4.0]).reshape(2, 1)

    # Create vector for parameter history.
    a_hist = np.zeros((2, n_epochs*N_train+1))
    # Initialize history vector.
    a_hist[:, 0] = a.reshape(2,)

    # Create vector to store eta history.
    alpha_hist = np.zeros((n_epochs*N_train))

    # Create array for storing training error values.
    Jgd = np.zeros(n_epochs*N_train+1)

    # Create array for storing test error values.
    Jgd_test = np.zeros(n_epochs*N_train+1)

    # Calcule o MSE com conjunto de treinamento para o conjunto de pesos inicial.
    Jgd[0] = (1.0/N_train)*np.sum(np.power(y_train - X_train.dot(a), 2))

    # Calcule o MSE com conjunto de teste para o conjunto de pesos inicial.
    Jgd_test[0] = (1.0/N_test)*np.sum(np.power((y_test - X_test.dot(a)), 2))

    # Cria arrays para armazenar vetores de atualização e gradiente.
    update_hist = np.zeros((2, n_epochs*N_train))
    gradient_hist = np.zeros((2, n_epochs*N_train))

    # Stocastic gradient-descent loop.
    iteration = 0
    min_test_error = float('inf')
    min_test_error_iteration = 0
    a_min = a
    # Época de treinamento, apresenta todas os exemplos de treinamento ao modelo.
    for epoch in range(n_epochs):

        # Shuffle the whole dataset before every epoch.
        shuffled_data_set_indexes = random.sample(range(0, N_train), N_train)

        # Iteração de treinamento, apenas um exemplo é apresentado ao modelo.
        for i in range(N_train):
            # Retrieve one pair of atribute vector and label.
            random_index = shuffled_data_set_indexes[i]
            xi = X_train[random_index:random_index+1]
            yi = y_train[random_index:random_index+1]

            # Decaimento temporal do passo de aprendizagem.
            alpha = timeBasedDecay(alpha_init, k, epoch*N_train + i)

            # Cálculo da estimativa do vetor gradiente com apenas uma amostra.
            gradient = -2.0*xi.T.dot(yi - xi.dot(a))
            update = alpha*gradient
            a = a - update

            # Armazena o histórico de valores.
            a_hist[:, epoch*N_train+i+1] = a.reshape(2,)
            alpha_hist[epoch*N_train+i] = alpha
            update_hist[:, epoch*N_train+i] = update.reshape(2,)
            gradient_hist[:, epoch*N_train+i] = gradient.reshape(2,)

            # Calcula o MSE com conjunto de treinamento por itereção de treinamento.
            Jgd[epoch*N_train+i+1] = (1.0/N_train)*np.sum(np.power((y_train - X_train.dot(a)), 2))

            # Calcula o MSE com conjunto de teste por itereção de treinamento.
            Jgd_test[epoch*N_train+i+1] = (1.0/N_test)*np.sum(np.power((y_test - X_test.dot(a)), 2))

            # Early-stopping.
            if Jgd_test[epoch*N_train+i+1] < min_test_error:
                min_test_error = Jgd_test[epoch*N_train+i+1]
                min_test_error_iteration = epoch*N_train+i+1
                a_min = a

            # Incrementa o contador de iterações.
            iteration = epoch*N_train+i

    return a, a_min, Jgd, Jgd_test, a_hist, alpha_hist, update_hist, gradient_hist, iteration

2. Execute a célula de código abaixo para criar o conjunto de dados que será usado neste exercício.

+ A função objetivo utilizada neste exercício é dada por
$$y = 2.5x - \lfloor 2.5 x + 0.5 \rfloor,$$
onde $a_1=2.5$, $a_2=-1.0$ e $\lfloor . \rfloor$ é uma função matemática que arredonda um número real para o menor inteiro que não é maior que o número. Em outras palavras, ela "arredonda para baixo" em direção ao negativo infinito.

+ A função hipótese que utilizaremos tem o mesmo formato da função objetivo,
$$\hat{y} = \hat{a}_1 + \hat{a}_2 \lfloor 2.5 x + 0.5 \rfloor,$$
sendo o objetivo do algoritmo do gradiente descendente estocástico encontrar aproximações, $\hat{a}_1$ e $\hat{a}_2$, para os valores ideais, ${a}_1$ e ${a}_2$.

+ Para representarmos a função hipótese em formato matricial, i.e., $\textbf{y} = \textbf{X}\textbf{a}$, precisamos criar a matriz de atributos concatenando os vetores dos atributos $\textbf{x}$ e $\lfloor 2.5 \textbf{x} + 0.5 \rfloor$.

**DICAS**

+ Na célula de código abaixo, o vetor do atributo $\textbf{x}$ é concatenado ao vetor do atributo $\lfloor 2.5 \textbf{x} + 0.5 \rfloor$, formando a matriz de atributos, $\textbf{X}$.
+ Essa concatenação é feita de forma manual, pois a implementação da versão estocática do gradiente descendente fornecida acima não faz isso automaticamente como no caso das classes fornecidas pela bilbioteca SciKit-Learn.

In [ ]:
# Número de amostras
N = 1000

# Gera um vetor de atributo.
x = np.linspace(0, 1, N, endpoint=False).reshape(N,1)

# Cria uma função dente de serra.
y = 2.5*x - np.floor(2.5*x + 0.5)

# Ruído.
w = np.sqrt(0.01)*np.random.randn(N, 1)

# Função observável.
y_noisy = y + w

# Cria matriz de atributos.
X = np.c_[x, np.floor(2.5*x + 0.5)]

# Figura comparando as duas funções.
plt.plot(x, y_noisy, label='Função observável')
plt.plot(x, y, label='Função objetivo')
plt.xlabel('x', fontsize=14)
plt.ylabel('y', fontsize=14)
plt.grid()
plt.legend()
plt.show()

3. Analise a geração das amostras da função observável no item anterior e responda: qual é o menor erro (i.e., erro quadrático médio - EQM) possível com um regressor linear treinado com essas amostras?

**Resposta**

<span style="color:blue">Digite abaixo a resposta do exercício.</span>

4. Divida o conjunto total de amostras em conjuntos de treinamento e validação. O conjunto de treinamento deve conter 75% do total de amostras e o conjunto de validação os 25% restantes.

**DICAS**

+ Use a função `train_test_split` e a configure com os seguintes parâmetros `test_size=0.25` e `random_state=seed`. A função divide o conjunto original de amostras em dois subconjuntos, um para treinamento e outro para validação (i.e., para avaliar a capacidade de generalização do modelo). Veja o código abaixo.
```python
X_train, X_test, y_train, y_test = train_test_split(X, y_noisy, test_size=0.25, random_state=seed)
```
+ Para que o próximo item do exercício funcione, chame as matrizes de treinamento e de validação de `X_train` e `X_test`, respectivamente, e os vetores de rótulos de treinamento e de validação de `y_train` e `y_test`, respectivamente, como no exemplo acima.

In [ ]:
# Digite o código do exercício aqui.

5. Execute a célula de código abaixo e analise as figuras.

A célula abaixo treina o modelo de regressão usando a função `gradientDescent` com os seguintes valores:
+ **taxa de decaimento ($k$)**: 0.1, 0.05, e 0.01.
+ **passo de aprendizagem ($\alpha$)**: 0.495, 0.25, 0.125, 0.0625 e 0.03125.

Cada figura mostra o erro de treinamento em função das iterações de treinamento para um valor específico da taxa de decaimento ($k$) e vários valores para o passo de aprendizagem ($\alpha$). O valor de taxa de decaimento ($k$) é mostrado no título (i.e., topo) da figura, enquanto os diferentes valores de passo de aprendizagem ($\alpha$) são mostrados com cores diferentes na legenda de cada figura.

**DICA**:

+ Lembrem-se que o menor valor do EQM tende ao valor da variância do ruído adicionado às amostras da função objetivo quando encontra-se os valores ótimos dos pesos.

In [ ]:
# Número de épocas.
n_epochs = 2
# Lista de taxas de decaimento.
k_list = [0.1, 0.05, 0.01]
# Lista de passos de aprendizagem.
alpha_list = [0.495, 0.25, 0.125, 0.0625, 0.03125]

# Lista para armazenar os erros das combinações de taxa de decaimento e passo de aprendizagem.
error = []
for k in k_list:
    error_hist = []
    for alpha in alpha_list:
        a, a_min, Jgd, Jgd_test, a_hist, alpha_hist, update_hist, gradient_hist, iteration = gradientDescent(X_train, y_train, X_test, y_test, n_epochs, alpha_init=alpha, k=k)
        error_hist.append(Jgd_test)
    error.append(error_hist)

# Visualização do erro durante o treinamento de cada passo de aprendizagem.
plt.figure(figsize=(15,5))
for i in range(len(k_list)):
    plt.subplot(1, 3, i+1)
    plt.title('k = '+str(k_list[i]))
    for j in range(len(alpha_list)):
        plt.plot(np.arange(error[i][j].shape[0]), error[i][j], label=('alpha = '+f'{alpha_list[j]}'))
        plt.yscale('log')
    plt.xlabel('Iterações')
    plt.ylabel('EQM')
    plt.legend()
    plt.grid()
    plt.ylim([0.008, 50])
plt.show()

6. Analise as figuras do item anterior e responda: Quais são os valores ideais para a taxa de decaimento ($k$) e o passo de aprendizagem ($\alpha$)? (**Justifique sua resposta**).

**DICA**

+ A ideia é que o aprendizado seja rápido, ou seja, convirja rapidamente (erro praticamente constante), mas sem muita oscilação no erro.

**Resposta**

<span style="color:blue">Digite abaixo a resposta do exercício.</span>

7. De posse dos valores ideais para a taxa de decaimento ($k$) e passo de aprendizagem ($\alpha$), treine novamente o modelo com estes valores e imprima os erros quadráticos médios (EQMs) obtidos para os conjuntos de treinamento de validação e o valor dos pesos $\hat{a}_1$ e $\hat{a}_2$.

**DICAS**

+ Configure a função `gradientDescent` com os melhores valores para a taxa de decaimento ($k$) e passo de aprendizagem ($\alpha$) obtidos no item anterior.
+ Os parâmetros de entrada da função `gradientDescent` são descritos em seu cabeçalho. Veja a definição da função.
+ Treine o modelo com o conjunto de treinamento.
+ Configure o **número de épocas**, `n_epochs`, com o valor `2`, ou seja, o modelo será treinado por 2 épocas.
+ Lembre-se que a função hipótese é expressa no formato vetorial como $\hat{\textbf{y}}=\textbf{X}\textbf{a}$, onde $\textbf{X}$ é a matriz de atributos e $\textbf{a}$ é o vetor de pesos. Portanto, para fazer predições com as matrizes de atributos de treinamento e validação, você precisa utilizar a função hipótese no formato vetorial.
+ Você pode usar a função `mean_squared_error` da biblioteca SciKit-Learn para calcular o EQM.

In [ ]:
# Digite o código do exercício aqui.

8. Treine um modelo usando a **equação normal** (i.e., a equação que dá a solução ótima para o conjunto de treinamento fornecido). Ao final, imprima o erro quadrático médio (EQM) obtido pelo modelo para os conjuntos de treinamento e validação. Além disso, imprima o valor dos pesos $\hat{a}_1$ e $\hat{a}_2$ obtidos com a **equação normal**.

**DICAS**

+ Você pode utilizar a classe `LinearRegression` da biblioteca SciKit-Learn para resolver este item ou implementar a equação normal manualmente.
+ Caso você use a classe `LinearRegression`, a configure com o parâmetro `fit_intercept=False`, pois a matriz de atributos criada no item 2 do exercício, já contém a coluna do atributos de bias, ou seja, a coluna com todos os valores iguais a 1.
+ Usando a classe `LinearRegression`:
  * A predição é feita com o método `predict()`.
  * Os pesos do modelo podem ser acessados através do atributo `coef_` da classe `LinearRegression`. Por exemplo, dado que o nome do objeto da classe `LinearRegression` é `reg`, então `reg.coef_[0,0]` acessa o valor ótimo encontrado para o peso $\hat{a}_1$ e `reg.coef_[0,1]` acessa o valor ótimo encontrado para o peso $\hat{a}_2$.
+ Você pode usar a função `mean_squared_error` da biblioteca SciKit-Learn para calcular o EQM.

In [ ]:
# Digite o código do exercício aqui.

9. Compare os pesos ($\hat{a}_1$ e $\hat{a}_2$) e os erros, i.e., EQMs, (para os conjuntos de treinamento e validação) obtidos com os modelos usando a equação normal (item 8) e o gradiente descendente estocástico com os melhores valores para a taxa de decaimento e passo de aprendizagem (item 7). Em seguida, responda: os valores dos pesos são diferentes? Se sim, explique o motivo da diferença. (**Justifique sua resposta**).

**DICAS**

+ Lembre-se que a equação normal dá a solução ótima, ou seja, ela fornece os pesos que minimizam o EQM. Não existem outros pesos que resultem em um EQM menor para o conjunto de treinamento usado.
+ As estimativas do vetor gradiente com o gradiente descendente estocástico, mesmo com os melhores valores para a taxa de decaimento e passo de aprendizagem, continuam sendo ruidosas, consequentemente, as atualizações dos pesos também serão ruidosas.
+ Além disso, os valores encontrados para a taxa de decaimento e passo de aprendizagem podem não ser os ótimos.
+ Reveja o material de aula e os exemplos onde discutimos as versões do gradiente descendente.

<span style="color:blue">Digite aqui a resposta do exercício.</span>

**Resposta**

10. Plote a superfície de contorno desta função hipótese e mostre que os pesos encontrados com a equação normal e gradiente descendente são próximos, mas não idênticos.

**DICAS**

+ Use a função `calculateErrorSurface` definida no item 1 deste exercício.
+ A função `calculateErrorSurface` restringe o eixo de $\hat{a}_1$ entre os valores $2.5$ e $2.59$.
+ A função `calculateErrorSurface` restringe o eixo de $\hat{a}_2$ entre os valores $-1.020$ e $-0.995$.
+ Use as funções `xlim` e `ylim` da biblioteca matplotlib para restringir a figura aos limites de $\hat{a}_1$ e $\hat{a}_2$ mencionados acima.

In [ ]:
# Digite o código do exercício aqui.

11. Plote uma figura que compare as funções objetivo, observável e aproximada (via gradiente descendente estocástico e equação normal). Use o conjunto total de amostras. No caso da função aproximada via gradiente descendente estocástico, use o vetor de pesos obtido com o `early-stopping`.

**DICAS**

+ Use a matriz $\textbf{X}$, criada no item 2, para fazer as predições com o conjunto total de dados.
+ Analise o código da função `gradientDescent` definida no item 1 para identificar qual é a variável com o vetor de pesos obtido com o `early-stopping`.

In [ ]:
# Digite o código do exercício aqui.

12. Discuta os resultados apresentados na figura anterior. O que você consegue concluir? (**Justifique sua resposta**)

**Resposta**

<span style="color:blue">Digite abaixo a resposta do exercício.</span>

### 2) Previsão do Desempenho de Hardware de Computadores

O conjunto de dados Computer Hardware contém informações sobre características técnicas de computadores de diferentes fabricantes, com o objetivo de estimar seu desempenho medido pelo ERP (Estimated Relative Performance). Sua tarefa será desenvolver um modelo de regressão para prever o ERP com base nas características técnicas dos sistemas. A tabela abaixo aprenta os atributos e o rótulo.


| Nome do Atributo | Descrição                                 | Tipo        | Unidade/Faixa                   |
|-------------------|-------------------------------------------|-------------|---------------------------------|
| **vendor**        | Fabricante do computador                  | Categórico  | Strings, 30 valores (ex: IBM, HP, Siemens) |
| **model** | Modelo | Categórico | Strings com os nomes dos modelos |
| **MYCT**          | Tempo do ciclo da máquina (Machine Cycle Time) | Numérico    | Nanossegundos (ns)             |
| **MMIN**          | Memória principal mínima                  | Numérico    | Kilobytes (KB)                 |
| **MMAX**          | Memória principal máxima                  | Numérico    | Kilobytes (KB)                 |
| **CACH**          | Memória cache                             | Numérico    | Kilobytes (KB)                 |
| **CHMIN**         | Número mínimo de canais de I/O            | Numérico    | Unidades                        |
| **CHMAX**         | Número máximo de canais de I/O            | Numérico    | Unidades                        |
| **PRP**           | *Published Relative Performance* (desempenho relativo publicado) | Numérico | Escala adimensional           |
| **ERP**           | **Rótulo (Target):** *Estimated Relative Performance* (desempenho relativo estimado) | Numérico | Escala adimensional |


1. Execute a célula de código abaixo para baixar e criar a base de dados.

In [ ]:
# Importando bibliotecas necessárias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Carregamento dos Dados
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/cpu-performance/machine.data"
columns = ['vendor', 'model', 'MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX', 'PRP', 'ERP']
df = pd.read_csv(url, header=None, names=columns)

# Imprime as 5 primeiras linhas da base de dados.
print(df.head())

# Criando a matriz de atributos, removendo as colunas vendor e model, pois são strings, e as colunas PRP e ERP, pois são os possíveis rótulos. Entretanto, usaremos apenas a coluna ERP como rótulo.
X = df.drop(["vendor", "model", "PRP", "ERP"], axis=1)
# Criando o vetor de rótulos.
y = df.ERP

2. Divida o conjunto de dados em 80% para validação e 20% para teste.

In [ ]:
# Digite o código do item aqui.

3. Padronize os conjuntos de treinamento e teste.

**DICAS**
+ Use a classe `StandardScaler`.
+ Não se esqueça que os parâmetros de padronização devem ser calculados com o conjunto de treinamento e aplicados aos conjuntos de treinamento e teste.

In [ ]:
# Digite o código do item aqui.

4. Treine um regressor linear com o conjunto de treinamento e calcule o erro quadrático médio com o conjunto de teste. Imprima o valor do erro.

**DICAS**
+ Use a classe `LinearRegression`.
+ Você pode usar a função `mean_squared_error` para calcular o erro.

In [ ]:
# Digite o código do item aqui.

5. Neste item, verificaremos se um regressor polinomial consegue apresentar um desempenho melhor, ou seja, apresentar um erro menor.

Portanto, usando a estratégia de validação cruzada **k-Fold**, encontre a ordem ideal para que uma função hipótese polinomial aproxime bem o conjunto de dados. Para avaliar qual é a ordem ideal para o polinômio aproximador, plote gráficos com a média e o desvio padrão do erro quadrático médio (EQM) em função dos graus de polinômio considerados. Para isso:

   1. Use o **k-Fold** com **k** igual a 5.
   2. Configure o parâmetro `shuffle` da classe `KFold` como `True`, ou seja, `shuffle=True`.
   3. Faça a análise de polinômios de ordem 1 até 7, **inclusive**.
   4. Desabilite a inclusão da coluna do atributo de bias ao instanciar a classe `PolynomialFeatures` utilizando o parâmetro `include_bias=False`.
   5. Use a classe `StandardScaler` para padronizar os atributos.
   6. Use todo o conjunto de dados, ou seja, `X` e `y`, para realizar a validação cruzada.

**DICAS**

+ Crie um pipeline de ações com objetos das classes `PolynomialFeatures`, `StandardScaler` e `LinearRegression`.
+ O tempo de execução desse exercício é de aproximadamente 10 minutos, mas pode variar de computador para computador, portanto, pegue um café e tenha paciência.
+ Para resolver este item, se baseie no seguinte exemplo: [validacao_cruzada.ipynb](https://colab.research.google.com/github/zz4fap/t319_aprendizado_de_maquina/blob/main/notebooks/regression/validacao_cruzada.ipynb).
+ **Atenção, não basta apenas copiar o código do exemplo dado, você precisa alterá-lo.**

In [ ]:
# Digite o código do item aqui.

6. Após analisar os resultados obtidos com a validação cruzada **k-Fold**, responda qual é a melhor ordem de polinômio para aproximar os dados. **Justifique sua resposta.**

**DICA**

* Lembre-se do princípio da navalha de Occam para escolher a melhor ordem.

**Resposta**

<span style="color:blue">Digite abaixo a resposta do exercício.</span>

7. De posse da melhor ordem, treine um novo modelo considerando esta ordem e ao final imprima o valor do erro quadrático médio para o conjunto de teste.

**DICAS**

+ Treine com o conjunto de treinamento e cacule o erro com o conjunto de teste.
+ Desabilite a inclusão da coluna do atributo de bias ao instanciar a classe `PolynomialFeatures` utilizando o parâmetro `include_bias=False`.
+ Use a classe `StandardScaler` para normalizar os dados.

In [ ]:
# Digite o código do item aqui.

8. O erro obtido no item anterior é menor do que o erro obtido no item 4 deste exercício? **Justifique sua resposta**.

**Resposta**

<span style="color:blue">Digite abaixo a resposta do exercício.</span>

9. Neste item, iremos remover valores discrepantes dos atributos. Valores discrepantes podem distorcer modelos de regressão (e.g., aumentar o erro quadrático médio), especialmente em datasets pequenos.

Para identificar e remover tais valores, usaremos o Intervalo Interquartil (IQR). O IQR é uma medida estatística robusta para identificar valores discrepantes. Valores fora do intervalo $[Q1 - 1.5*IQR, Q3 + 1.5*IQR]$ são considerados discrepantes. A remoção de valores discrepantes melhora a generalização do modelo e reduz o impacto de valores extremos em algoritmos sensíveis (e.g., regressão).

Execute a célula de código abaixo para remover os valores discrepantes.

**DICAS**
+ Além de remover os valores discrepantes, o código abaixo cria uma nova base de daos, sem tais valores, e a divide em novos conjuntos de treinamento e validação.

In [ ]:
# Selecionar features numéricas
numeric_features = ['MYCT', 'MMIN', 'MMAX', 'CACH', 'CHMIN', 'CHMAX']

df = df.drop(["vendor", "model", "PRP"], axis=1)

# Calcular IQR para cada feature
Q1 = df[numeric_features].quantile(0.25)
Q3 = df[numeric_features].quantile(0.75)
IQR = Q3 - Q1

# Definir limites
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filtrar dados
df_clean = df[
    ~((df[numeric_features] < lower_bound) | (df[numeric_features] > upper_bound)).any(axis=1)
]

# Criando um novo dataset, mas desta vez sem amostras discrepantes.
X = df_clean.drop(["ERP"], axis=1)
y = df_clean.ERP

# Divide o novo conjunto em conjuntos de treinamento e validação.
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size = 0.2, random_state=42)

# Criar figura com 2 subplots lado a lado
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Plotar boxplot antes da remoção (subplot esquerdo)
sns.boxplot(data=df[numeric_features], ax=axes[0])
axes[0].set_title("Antes da Remoção de Outliers")
axes[0].tick_params(axis='x', rotation=45)  # Rotacionar labels se necessário

# Plotar boxplot após remoção (subplot direito)
sns.boxplot(data=df_clean[numeric_features], ax=axes[1])
axes[1].set_title("Após Remoção de Outliers (IQR)")
axes[1].tick_params(axis='x', rotation=45)

# Ajustar layout e mostrar
plt.tight_layout()
plt.show()

10. De posse dos novos conjuntos de treinamento e teste sem valores discrepantes e da melhor ordem encontrada no item 5 deste exercício, treine um novo modelo considerando os novos conjuntos e esta ordem e ao final imprima o valor do erro quadrático médio para o conjunto de teste.

**DICAS**

+ Treine com o conjunto de treinamento e cacule o erro com o conjunto de teste.
+ Desabilite a inclusão da coluna do atributo de bias ao instanciar a classe `PolynomialFeatures` utilizando o parâmetro `include_bias=False`.
+ Use a classe `StandardScaler` para normalizar os dados.

In [ ]:
# Digite o código do item aqui.

11. O erro obtido no item anterior é menor do que o erro obtido no item 7 deste exercício? **Justifique sua resposta**.

**Resposta**

<span style="color:blue">Digite abaixo a resposta do exercício.</span>

### 3) Usando regressão para estimar calorias queimadas.

Neste exercício, você utilizará uma técnica de **validação cruzada** para encontrar um modelo que estime a quantidade de calorias queimadas após uma atividade física com base em um conjunto de dados coletados. As informações das **colunas** contidas no conjunto de dados seguem abaixo. O objetivo é utilizar os atributos para estimar a quantidade de calorias queimadas.

|            |                   **Atributos**                   |
|:----------:|:-------------------------------------------------:|
|   User_ID  |              Identificação do usuário             |
|   Gender   |                       Gênero                      |
|     Age    |                       Idade                       |
|   Height   |                       Altura                      |
|   Weight   |                        Peso                       |
|  Duration  |            Duração da atividade física            |
| Heart_Rate | Média de batimentos cardíacos durante a atividade física |
|  Body_Temp | Média da temperatura coporal durante a atividade física |
|            |                     **Rótulo**                    |
|  Calories  |       Calorias queimadas durante a atividade física       |

1. Execute a célula de código abaixo para importar o conjunto de dados e as bibliotecas necessárias.

**DICAS**

+ Após a execução bem sucedida da célula abaixo, você visualizará as 5 primeiras linhas do arquivo.

In [ ]:
# Importe todas as bibliotecas necessárias.
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
import urllib

# Reseta o gerador de sequências pseudo aleatórias.
seed = 42
np.random.seed(seed)

# Baixa as bases de dados do dropbox.
urllib.request.urlretrieve('https://www.dropbox.com/s/1zka46bw4f4z5xq/exercise.csv?dl=1', 'exercise.csv')
urllib.request.urlretrieve('https://www.dropbox.com/s/45gtml94o97bhz8/calories.csv?dl=1', 'calories.csv')

# Importa os arquivos CSV.
exercise_data = pd.read_csv('./exercise.csv')
calories_data = pd.read_csv('./calories.csv')

# Une as duas bases de dados.
df = exercise_data.join( calories_data.set_index('User_ID'), on='User_ID', how='left')

# Mostra uma tabela com as 5 primeiras linhas.
df.head()

2. Execute a célula de código abaixo para aplicar um pré-processamento aos dados do conjunto.

+ Como os modelos de regressão esperam valores numéricos, devemos alterar os valores textuais da coluna `Gender` em valores numéricos. A string `male` é alterada para o valor 0 e a string `female` é alterada para o valor 1.
+ Na sequência, a coluna `User_ID` é removida, pois ela não é um atributo e, portanto, não traz informação útil para a regressão.

In [ ]:
# Mapeia as strings em valores numéricos.
df.replace({'Gender':{'male':0, 'female':1}}, inplace=True)

# Remove a coluna 'User_ID'.
del df[ 'User_ID' ]

# Mostra uma tabela com as 5 primeiras linhas.
df.head()

3. Execute a próxima célula de código abaixo para criar a matriz de atributos, $\textbf{X}$, e o vetor de rótulos, $\textbf{y}$.

**DICAS**

+ A primeira linha de comando remove da matriz de atributos a coluna `Calories`, pois ela será nosso rótulo.
+ A segunda linha cria o vetor de rótulos contendo apenas a coluna `Calories`.
+ A célula imprimirá as dimensões da matriz de atributos e do vetor de rótulos.

In [ ]:
# Criando a matriz de atributos e o vetor de rótulos.
X = df.drop('Calories', axis=1)
y = df['Calories']

# Atributos.
print('Dimensão da matriz de atributos:', X.shape)
# Rótulos.
print('Dimensão do vetor de rótulos:',y.shape)

4. Com a matriz de atributos, $\textbf{X}$, e o vetor de rótulos, $\textbf{y}$, obtidos no item anterior, utilize a técnica de validação cruzada k-Fold para escolher a melhor ordem para um modelo de regressão polinomial.

Para isso, faça o seguinte:

1. Use o **k-Fold** instanciado com os seguintes parâmetros `n_splits=10`, `shuffle=True` e `random_state=seed`.
2. Faça a análise de **polinômios** de ordem 1 até 7, **inclusive**.
3. Para realizar a validação cruzada com o **k-Fold**, use a função `cross_val_score` com o seguinte parâmetro `scoring='neg_mean_squared_error'`.
4. Use a classe `StandardScaler` para padronizar os dados.
5. Use a classe `LinearRegression` para realizar a regressão propriamente dita.
6. Plote gráficos com a média e o desvio padrão do erro quadrático médio em função do grau do polinômio.

**DICAS**

+ O tempo de execução desse exercício é de aproximadamente 10 minutos, mas pode variar de computador para computador, portanto, pegue um café e tenha paciência.
+ Use o princípio da navalha de Occam para escolher a ordem do polinômio.
+ Para resolver este item, se baseie no seguinte exemplo: [validacao_cruzada.ipynb](https://colab.research.google.com/github/zz4fap/t319_aprendizado_de_maquina/blob/main/notebooks/regression/validacao_cruzada.ipynb).
+ **Atenção, não basta apenas copiar o código do exemplo dado, você precisa alterá-lo.**

In [ ]:
# Digite o código do item aqui.

5. Após analisar os resultados do item anterior responda: Qual a melhor ordem do polinômio para esse problema? **Justifique sua resposta**.

**Resposta**

<span style="color:blue">Digite abaixo a resposta do exercício.</span>

6. De posse da melhor ordem, treine um novo modelo considerando esta ordem e no final imprima o valor do erro quadrático médio (MSE) para os conjuntos de treinamento e de validação.

Para isso, faça o seguinte

1. Separe 75% do conjunto de dados para o treinamento e 25% para o conjunto de validação com o parâmetro `random_state=seed`.
2. Crie um pipeline com as seguintes ações:
    + `PolynomialFeatures` com a ordem escolhida.
    + `StandardScaler` para padronizar os dados.
    + `LinearRegression` para encontrar os pesos da função hipótese polinomal.
3. Treine o modelo com o conjunto de treinamento.
4. Faça predições com o modelo treinando usando os conjuntos de treinamento e validação.
5. Calcule e imprima o MSE entre as predições feitas pelo modelo e os rótulos dos conjuntos de treinamento e validação.

**DICAS**

+ Para resolver este item, se baseie no seguinte exemplo: [validacao_cruzada.ipynb](https://colab.research.google.com/github/zz4fap/t319_aprendizado_de_maquina/blob/main/notebooks/regression/validacao_cruzada.ipynb).
+ **Atenção, não basta apenas copiar o código do exemplo dado, você precisa alterá-lo.**

In [ ]:
# Digite o código do item aqui.

7. Comparando os dois erros obtidos no item anterior, erros de treinamento e validação. Você diria que o modelo está subajustando, sobreajustando ou encontrou uma relação de compromisso entre generalização e flexibilidade (ou capacidade)? **Justifique sua resposta**.

**Resposta**

<span style="color:blue">Digite abaixo a resposta do exercício.</span>

8. Treine um novo modelo considerando uma ordem igual a 10 e no final imprima o valor do erro quadrático médio (MSE) para os conjuntos de treinamento e de validação.

Para isso, faça o seguinte

1. Separe 75% do conjunto de dados para o treinamento e 25% para o conjunto de validação com o parâmetro `random_state=seed`.
2. Crie um pipeline com as seguintes ações:
    + `PolynomialFeatures` com a ordem escolhida.
    + `StandardScaler` para padronizar os dados.
    + `LinearRegression` para encontrar os pesos da função hipótese polinomal.
3. Treine o modelo com o conjunto de treinamento.
4. Faça predições com o modelo treinando usando os conjuntos de treinamento e de validação.
5. Calcule e imprima o MSE entre as predições feitas pelo modelo e os rótulos dos conjuntos de treinamento e validação.

**DICAS**

+ Para resolver este item, se baseie no seguinte exemplo: [validacao_cruzada.ipynb](https://colab.research.google.com/github/zz4fap/t319_aprendizado_de_maquina/blob/main/notebooks/regression/validacao_cruzada.ipynb).
+ **Atenção, não basta apenas copiar o código do exemplo dado, você precisa alterá-lo.**

In [ ]:
# Digite o código do item aqui.

9. Comparando os dois erros obtidos no item anterior, erros de treinamento e de validação. Você diria que o modelo está subajustando, sobreajustando ou encontrou uma relação de compromisso entre capacidade de generalização e flexibilidade (ou capacidade)? **Justifique sua resposta**.

**Resposta**

<span style="color:blue">Digite abaixo a resposta do exercício.</span>